In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9681,2024-10-16,Europa Euroliga,15:30,Barcelona,Monaco,1.61,2.42,161.5,1.91,1.91,-4.5,2.00,1.71,QDlTiVZ8,0.621118,0.413223,0.523560,0.523560,0.034341,0.2,0.4,NaN,NaN,130.342,58.380957,0.447906,148.002,35.111458,0.237236,103.504,165.332,94.16,205.92,0.0,0.0,0.0,0.0,0.284246,0.000000,0.110545,-1.35,-0.270,-2.259259,0.714849,0.8,0.085151,2.68,0.536,2.649254,0.511402,0.5,-0.011402
9682,2024-10-16,Europa Eurocopa,13:00,Bahcesehir Kol.,Trento,1.21,4.08,158.5,1.85,1.85,-10.5,1.98,1.74,jarEfDEb,0.826446,0.245098,0.540541,0.540541,0.071544,0.6,0.8,NaN,NaN,148.246,36.607690,0.246939,193.102,135.110292,0.699684,134.804,327.048,201.84,424.44,0.0,0.0,0.0,0.0,0.767258,0.000000,0.091240,2.01,0.402,0.522388,0.000000,0.0,0.000000,0.93,0.186,16.559140,0.000000,0.0,0.000000
9683,2024-10-16,Europa Eurocopa,14:00,Buducnost,Trefl Sopot,1.28,3.44,159.5,1.85,1.85,-8.5,1.93,1.78,rZv6dZqn,0.781250,0.290698,0.540541,0.540541,0.071948,0.4,0.0,NaN,NaN,208.726,14.369321,0.068843,NaN,NaN,NaN,182.460,0.000,215.00,262.86,0.0,0.0,0.0,0.0,0.647182,0.000000,0.057178,0.06,0.012,23.333333,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9684,2024-10-16,Europa Eurocopa,15:30,Joventut,BC Wolves,1.17,4.66,166.5,1.87,1.83,-11.5,2.00,1.72,xdG1o9yo,0.854701,0.214592,0.534759,0.546448,0.069293,0.6,0.2,NaN,NaN,149.250,36.998396,0.247895,289.002,261.800557,0.905878,128.064,394.758,197.34,718.20,0.0,1.0,0.0,0.0,0.846588,0.015289,0.106446,1.41,0.282,0.602837,0.000000,0.0,0.000000,2.47,0.494,7.408907,0.000000,0.0,0.000000
9685,2024-10-16,Ásia Easl,07:10,Hiroshima D.,Hong Kong Eastern,1.18,4.47,161.5,1.88,1.78,-9.5,1.97,1.71,QyTxQmWP,0.847458,0.223714,0.531915,0.561798,0.071171,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.823498,0.038640,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,2024-10-16,Itália Série A2,15:30,Orzinuovi,Rimini,1.95,1.72,158.5,1.83,1.83,1.5,1.83,1.83,CbUjrvPk,0.512821,0.581395,0.546448,0.546448,0.094216,0.8,0.8,NaN,NaN,164.874,37.158987,0.225378,148.520,45.757140,0.308087,182.212,211.022,112.23,130.83,0.0,0.0,0.0,0.0,0.088629,0.000000,0.000000,3.25,0.650,1.461538,0.470458,0.5,0.029542,4.52,0.904,0.796460,0.000000,0.0,0.000000
9769,2024-10-16,Itália Série A2,15:30,Verona,Cantu,1.56,2.21,150.5,1.82,1.84,-4.5,1.95,1.72,j5HIdwvq,0.641026,0.452489,0.549451,0.543478,0.093514,0.2,0.4,NaN,NaN,168.082,75.593607,0.449742,127.114,39.496946,0.310721,113.710,135.208,120.54,115.52,0.0,0.0,0.0,0.0,0.243830,0.007728,0.088629,1.26,0.252,2.222222,0.750776,0.8,0.049224,-3.05,-0.610,-1.983607,0.632257,0.5,-0.132257
9770,2024-10-16,Mundo Amistoso Interclubes,23:00,Portland Trail Blazers (Usa),Ulm (Ger),1.06,30.00,204.5,1.83,1.87,-37.5,1.95,1.76,UFWRkFSi,0.943396,0.033333,0.546448,0.534759,-0.023270,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.317686,0.015289,0.072426,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9771,2024-10-16,Rússia Superliga,10:00,Temp Sumz Revda,Zenit Petersburg 2,1.35,3.02,152.5,1.87,1.83,-7.5,1.91,1.76,jToSy8FK,0.740741,0.331126,0.534759,0.546448,0.071867,0.0,0.2,NaN,NaN,108.280,24.494088,0.226211,164.402,56.628317,0.344450,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
